In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import xgboost as xgb
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import datetime
from sklearn.cross_validation import train_test_split
import operator


0it [00:00, ?it/s]


In [95]:
train = pd.read_csv('./train_1.csv').fillna(0)

In [2]:
key = pd.read_csv('./key_trunc.csv')

In [3]:
sub = pd.read_csv("./sample_submission_1.csv")

In [2]:
train_melt = pd.read_csv("train_feats.csv")

In [3]:
for col in train.columns[1:]:
    train[col] = pd.to_numeric(train[col],downcast='integer')

NameError: name 'train' is not defined

In [3]:
def get_median(row, last_n_days):
    return pd.rolling_median(pd.Series(row), window = last_n_days)    

In [188]:
def get_mean(row, last_n_days):
    return pd.rolling_mean(pd.Series(row), window = last_n_days)    

In [189]:
def get_mean_diff(row, last_n_days):
    return pd.Series(pd.rolling_mean(np.diff(pd.Series(row)), window = last_n_days))

In [174]:
def get_last_day_visits(row):
    return row[1:]

In [192]:
def get_long_stats(func_type, df, df_melt, last_n_days = None):

    if func_type == "median":
        rolling_stats = df.iloc[:, -60:].progress_apply(get_median, last_n_days = last_n_days, axis = 1)
        
    elif func_type == "mean":
        rolling_stats = df.iloc[:, -60:].progress_apply(get_mean, last_n_days = last_n_days, axis = 1)
        
    elif func_type == "mean_diff":
        rolling_stats = df.iloc[:, -61:].progress_apply(get_mean_diff, last_n_days = last_n_days, axis = 1)
        
    elif func_type == "last_day_visits":        
        rolling_stats = df.iloc[:, -61:].progress_apply(get_last_day_visits, axis = 1)

    rolling_stats_df = pd.concat([df.Page, rolling_stats.apply(pd.Series)], axis = 1)  
    rolling_stats_df.columns = ["Page"] + list(df.columns[-60:])
    
    rolling_stats_df = pd.melt(rolling_stats_df, id_vars=['Page'], var_name="date", value_name=func_type + "_" + str(last_n_days))        
    
    df_melt = pd.merge(df_melt, rolling_stats_df, on = ["Page", "date"])     
    
    return df_melt

In [193]:
def generate_train_melt():

#     train_melt = pd.read_csv("train_feats.csv")

#     Commented because these features are already generated
    train_melt = pd.melt(pd.concat([train.Page, train.iloc[:, -60:]], axis = 1), id_vars=['Page'], var_name="date", value_name="visits")

    train_melt = get_long_stats("mean", train, train_melt, 7)
    train_melt = get_long_stats("mean", train, train_melt, 30)
    train_melt = get_long_stats("mean", train, train_melt, 60)

    train_melt = get_long_stats("median", train, train_melt, 7)
    train_melt = get_long_stats("median", train, train_melt, 30)
    train_melt = get_long_stats("median", train, train_melt, 60)

    train_melt = get_long_stats("mean_diff", train, train_melt, 7)
    train_melt = get_long_stats("last_day_visits", train, train_melt)
    
    train_melt["date_num"] = train_melt.progress_apply(get_date, axis = 1)
    train_melt["weekday"] = train_melt.progress_apply(get_weekday, axis = 1)
    
    # Columns i am not using
    # train_melt["month"] = train_melt.progress_apply(get_month, axis = 1)
    # train_melt["language"] = train_melt.progress_apply(get_language, axis = 1)
    
    return train_melt 

train_melt = generate_train_melt()
train_melt.to_csv("train_feats.csv", index = False)



















  0%|          | 0/145063 [00:00<?, ?it/s]/Library/Python/2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pd.rolling_mean is deprecated for ndarrays and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


















  0%|          | 76/145063 [00:00<03:11, 756.52it/s]

















  0%|          | 305/145063 [00:00<02:32, 946.42it/s]

















  0%|          | 522/145063 [00:00<02:06, 1139.04it/s]

















  1%|          | 727/145063 [00:00<01:49, 1314.16it/s]

















  1%|          | 959/145063 [00:00<01:35, 1510.14it/s]

















  1%|          | 1179/145063 [00:00<01:26, 1665.69it/s]

















  1%|          | 1405/145063 [00:00<01:19, 1808.13it/s]

















  1%|          | 1614/145063 [00:00<01:16, 1882.76it/s]
















  2%|▏         | 2372/145063 [03:01<3:02:24, 13.04it/s]

















  1%|▏         | 1816/145063 [00:00

 10%|▉         | 13898/145063 [00:06<00:57, 2296.45it/s]

















 10%|▉         | 14128/145063 [00:06<01:01, 2141.87it/s]

















 10%|▉         | 14373/145063 [00:06<00:58, 2224.41it/s]

















 10%|█         | 14611/145063 [00:06<00:57, 2267.19it/s]

















 10%|█         | 14840/145063 [00:06<00:57, 2261.08it/s]

















 10%|█         | 15078/145063 [00:06<00:56, 2294.07it/s]

















 11%|█         | 15309/145063 [00:06<00:56, 2296.37it/s]

















 11%|█         | 15555/145063 [00:07<00:55, 2342.46it/s]

















 11%|█         | 15790/145063 [00:07<00:56, 2302.91it/s]

















 11%|█         | 16047/145063 [00:07<00:54, 2376.38it/s]

















 11%|█         | 16286/145063 [00:07<00:55, 2323.97it/s]

















 11%|█▏        | 16533/145063 [00:07<00:54, 2365.20it/s]

















 12%|█▏        | 16771/145063 [00:07<00:54, 2358.44it/s]

















 12%|█▏        | 17008/145063 [00:07<0

 20%|██        | 29628/145063 [00:12<00:46, 2478.80it/s]

















 21%|██        | 29877/145063 [00:12<00:47, 2444.24it/s]

















 21%|██        | 30122/145063 [00:13<00:47, 2439.92it/s]

















 21%|██        | 30367/145063 [00:13<00:48, 2363.81it/s]

















 21%|██        | 30612/145063 [00:13<00:47, 2386.51it/s]

















 21%|██▏       | 30857/145063 [00:13<00:47, 2405.08it/s]

















 21%|██▏       | 31098/145063 [00:13<00:47, 2386.64it/s]

















 22%|██▏       | 31339/145063 [00:13<00:47, 2388.79it/s]

















 22%|██▏       | 31585/145063 [00:13<00:47, 2407.27it/s]

















 22%|██▏       | 31835/145063 [00:13<00:46, 2432.22it/s]

















 22%|██▏       | 32079/145063 [00:13<00:47, 2394.54it/s]

















 22%|██▏       | 32324/145063 [00:13<00:46, 2406.41it/s]

















 22%|██▏       | 32578/145063 [00:14<00:46, 2442.73it/s]

















 23%|██▎       | 32823/145063 [00:14<0

 31%|███       | 45274/145063 [00:19<00:41, 2411.44it/s]

















 31%|███▏      | 45516/145063 [00:19<00:58, 1700.13it/s]

















 32%|███▏      | 45761/145063 [00:19<00:53, 1871.37it/s]

















 32%|███▏      | 46005/145063 [00:19<00:49, 2011.13it/s]

















 32%|███▏      | 46256/145063 [00:20<00:46, 2136.99it/s]

















 32%|███▏      | 46494/145063 [00:20<00:44, 2204.23it/s]

















 32%|███▏      | 46746/145063 [00:20<00:42, 2289.64it/s]

















 32%|███▏      | 46985/145063 [00:20<00:43, 2249.94it/s]

















 33%|███▎      | 47217/145063 [00:20<00:43, 2270.43it/s]

















 33%|███▎      | 47468/145063 [00:20<00:41, 2335.68it/s]

















 33%|███▎      | 47706/145063 [00:20<00:41, 2329.26it/s]

















 33%|███▎      | 47964/145063 [00:20<00:40, 2397.00it/s]

















 33%|███▎      | 48207/145063 [00:20<00:40, 2369.70it/s]

















 33%|███▎      | 48446/145063 [00:21<0

 42%|████▏     | 60804/145063 [00:26<00:37, 2229.51it/s]

















 42%|████▏     | 61036/145063 [00:26<00:37, 2247.22it/s]

















 42%|████▏     | 61285/145063 [00:26<00:36, 2312.62it/s]

















 42%|████▏     | 61522/145063 [00:26<00:36, 2270.42it/s]

















 43%|████▎     | 61772/145063 [00:26<00:35, 2334.14it/s]

















 43%|████▎     | 62025/145063 [00:26<00:34, 2387.29it/s]

















 43%|████▎     | 62267/145063 [00:27<00:34, 2374.43it/s]

















 43%|████▎     | 62517/145063 [00:27<00:34, 2408.42it/s]

















 43%|████▎     | 62760/145063 [00:27<00:34, 2398.64it/s]

















 43%|████▎     | 63008/145063 [00:27<00:33, 2421.41it/s]

















 44%|████▎     | 63258/145063 [00:27<00:33, 2443.96it/s]

















 44%|████▍     | 63504/145063 [00:27<00:33, 2445.96it/s]

















 44%|████▍     | 63755/145063 [00:27<00:33, 2463.29it/s]

















 44%|████▍     | 64002/145063 [00:27<0

...

 73%|███████▎  | 105713/145063 [00:46<00:18, 2124.36it/s]

















 73%|███████▎  | 105926/145063 [00:46<00:18, 2108.62it/s]

















 73%|███████▎  | 106138/145063 [00:46<00:18, 2051.71it/s]

















 73%|███████▎  | 106352/145063 [00:46<00:18, 2073.93it/s]

















 73%|███████▎  | 106563/145063 [00:47<00:18, 2081.61it/s]

















 74%|███████▎  | 106772/145063 [00:47<00:18, 2040.45it/s]

















 74%|███████▎  | 106977/145063 [00:47<00:19, 1972.01it/s]

















 74%|███████▍  | 107175/145063 [00:47<00:19, 1902.56it/s]

















 74%|███████▍  | 107401/145063 [00:47<00:18, 1995.34it/s]

















 74%|███████▍  | 107612/145063 [00:47<00:18, 2026.19it/s]

















 74%|███████▍  | 107818/145063 [00:47<00:18, 2034.23it/s]

















 74%|███████▍  | 108023/145063 [00:47<00:18, 1954.39it/s]

















 75%|███████▍  | 108227/145063 [00:47<00:18, 1976.21it/s]

















 75%|███████▍  | 108439/1

 82%|████████▏ | 119381/145063 [00:53<00:11, 2216.58it/s]

















 82%|████████▏ | 119603/145063 [00:53<00:11, 2197.53it/s]

















 83%|████████▎ | 119823/145063 [00:53<00:11, 2168.82it/s]

















 83%|████████▎ | 120046/145063 [00:53<00:11, 2185.68it/s]

















 83%|████████▎ | 120269/145063 [00:53<00:11, 2198.29it/s]

















 83%|████████▎ | 120489/145063 [00:53<00:11, 2176.06it/s]

















 83%|████████▎ | 120708/145063 [00:53<00:11, 2179.78it/s]

















 83%|████████▎ | 120927/145063 [00:53<00:11, 2054.49it/s]

















 84%|████████▎ | 121135/145063 [00:53<00:11, 2039.58it/s]

















 84%|████████▎ | 121341/145063 [00:53<00:11, 2024.94it/s]

















 84%|████████▍ | 121567/145063 [00:54<00:11, 2086.85it/s]

















 84%|████████▍ | 121794/145063 [00:54<00:10, 2138.39it/s]

















 84%|████████▍ | 122009/145063 [00:54<00:10, 2140.39it/s]

















 84%|████████▍ | 122231/1

 93%|█████████▎| 134539/145063 [00:59<00:04, 2254.14it/s]

















 93%|█████████▎| 134767/145063 [00:59<00:04, 2223.38it/s]

















 93%|█████████▎| 134992/145063 [01:00<00:04, 2197.41it/s]

















 93%|█████████▎| 135215/145063 [01:00<00:04, 2199.93it/s]

















 93%|█████████▎| 135438/145063 [01:00<00:04, 2207.43it/s]

















 94%|█████████▎| 135660/145063 [01:00<00:04, 2177.42it/s]

















 94%|█████████▎| 135890/145063 [01:00<00:04, 2210.43it/s]

















 94%|█████████▍| 136112/145063 [01:00<00:04, 2183.46it/s]

















 94%|█████████▍| 136331/145063 [01:00<00:04, 2167.79it/s]

















 94%|█████████▍| 136549/145063 [01:00<00:03, 2152.33it/s]

















 94%|█████████▍| 136777/145063 [01:00<00:03, 2185.34it/s]

















 94%|█████████▍| 137016/145063 [01:00<00:03, 2240.71it/s]

















 95%|█████████▍| 137241/145063 [01:01<00:03, 2215.96it/s]

















 95%|█████████▍| 137464/1

KeyboardInterrupt: 

  4%|▍         | 331263/8703780 [00:30<03:22, 41305.30it/s]

In [7]:
def get_source(row):
    return row.Page.split["."][-1].split("_")[-1]

In [8]:
def get_language(row):
    try:
        return row.Page.split(".")[0].split("_")[-1]    
    except:
        return "en"

In [9]:
def get_month(row):
    return row.date.split("-")[1]

In [10]:
def get_date(row):
    return row.date.split("-")[2]

In [11]:
def get_weekday(row):
    return datetime.datetime.strptime(row.date, '%Y-%m-%d').date().weekday()

In [13]:
# def get_language_mean(row):
# lang_count = pd.Series(train_melt.groupby(["language"])["Page"].count()).sort_values(axis = 0, ascending = False)
# print lang_count

In [12]:
def get_median_7(row, last_index):
    
    return np.median(row.iloc[last_index - 7 : last_index].tolist())

In [13]:
def get_median_30(row, last_index):
    
    return np.median(row.iloc[last_index - 30 : last_index].tolist())

In [14]:
def get_median_60(row, last_index):
    
    return np.median(row.iloc[last_index - 60 : last_index].tolist())

In [15]:
def get_mean_7(row, last_index):
    
    return np.mean(row.iloc[last_index - 7 : last_index].tolist())

In [16]:
def get_mean_30(row, last_index):
    
    return np.mean(row.iloc[last_index - 30 : last_index].tolist())

In [17]:
def get_mean_60(row, last_index):
    
    return np.mean(row.iloc[last_index - 60 : last_index].tolist())

In [18]:
# def get_test_feats(row, last_index):
    
#     subset_row = row.iloc[last_index - 60 : last_index].tolist()
    
#     median_7 = np.median(subset_row[-7:])
#     median_30 = np.median(subset_row[-30:])
#     median_60 = np.median(subset_row[-60:])

#     mean_7 = np.mean(subset_row[-7:])
#     mean_30 = np.mean(subset_row[-30:])
#     mean_60 = np.mean(subset_row[-60:])
    
#     return pd.Series([row.Page, median_7, median_30, median_60, mean_7, mean_30, mean_60])

In [66]:
def smape(y_pred, y_true):
        
    y_true = y_true.get_label()
    y_pred = np.array(y_pred)
        
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0

    return "smape", np.nanmean(diff)

In [90]:
def run_xgb(train, label_train, valid = None, label_valid = None):

    # Set our parameters for xgboost
    params = {}
    params['objective'] = 'reg:linear'
    params['eta'] = 0.1
    params['max_depth'] = 6
    params['silent'] = 1
    params['min_child_weight'] = 0
    params['subsample'] = 1
    params['colsample_bytree'] = 1
    params['nthread'] = 13
    params['gamma'] = 0
    params['max_delta_step'] = 0

    d_train = xgb.DMatrix(train, label=label_train)
    
    if valid is not None:
        d_valid = xgb.DMatrix(valid, label=label_valid)
        watchlist = [(d_train, 'train'), (d_valid, 'validation')]
    else:
        watchlist = [(d_train, 'train')]
        
    bst = xgb.train(params, d_train, 200, watchlist, early_stopping_rounds=10, verbose_eval=10, feval=smape, maximize = False)
    
    return bst

In [23]:
label_train_melt = train_melt["visits"]

train_melt.drop("visits", axis = 1, inplace = True)
train_melt.drop("date", axis = 1, inplace = True)
train_melt.drop("month", axis = 1, inplace = True)
train_melt.drop("language", axis = 1, inplace = True)
train_melt.drop("Page", axis = 1, inplace = True)

In [91]:
x_train, x_valid, label_train, label_valid = train_test_split(train_melt, label_train_melt, test_size=0.2, random_state=4242)

bst = run_xgb(x_train, label_train, x_valid, label_valid)
# bst = run_xgb(train_melt, label_train_melt)

[0]	train-rmse:81076.6	validation-rmse:81130.6	train-smape:143.502	validation-smape:142.575
Multiple eval metrics have been passed: 'validation-smape' will be used for early stopping.

Will train until validation-smape hasn't improved in 10 rounds.
[10]	train-rmse:80501.6	validation-rmse:80922.4	train-smape:65.3812	validation-smape:65.9836
[20]	train-rmse:80344.4	validation-rmse:80898.1	train-smape:54.8142	validation-smape:55.2881
[30]	train-rmse:80291	validation-rmse:80899	train-smape:51.2666	validation-smape:51.7054
[40]	train-rmse:80271	validation-rmse:80903.5	train-smape:52.7021	validation-smape:53.5681
Stopping. Best iteration:
[33]	train-rmse:80282.4	validation-rmse:80900.6	train-smape:50.7296	validation-smape:51.2577



In [92]:
importance = bst.get_fscore()

importance = sorted(importance.items(), key=operator.itemgetter(1))

df_imp = pd.DataFrame(importance, columns=['feature', 'fscore'])
df_imp['fscore'] = df_imp['fscore']/df_imp['fscore'].sum()

print df_imp

     feature    fscore
0    mean_60  0.008529
1  median_60  0.009950
2    weekday  0.114428
3    mean_30  0.120824
4   date_num  0.125089
5  median_30  0.130775
6   median_7  0.235963
7     mean_7  0.254442


In [32]:
def get_preds(test_feats):
    
    d_test = xgb.DMatrix(test_feats)
    p_test = bst.predict(d_test)
    
    return pd.Series(p_test)

In [24]:
def get_overall_df():
    
#     key = pd.read_csv("key_trunc.csv")
    key_wide = pd.DataFrame()
    key_wide["page"] = np.unique(key.trunc_page).tolist()
    for i in np.unique(key.date).tolist():
        key_wide[i] = 0

    overall_df = pd.merge(train, key_wide, left_on="Page", right_on = "page", how = "inner")
    overall_df.drop("page", axis = 1, inplace = True)
    
    return overall_df

In [40]:
def test_and_pred(overall_df):
    
    pred_df = pd.DataFrame()
    
    for i in range(overall_df.columns.tolist().index('2017-01-01'), len(overall_df.columns)):

        last_date = overall_df.columns.tolist()[i]
        print last_date

        last_index = overall_df.columns.tolist().index(last_date)

        test_feats = pd.DataFrame()
        
        test_feats["Page"] = overall_df.Page
        test_feats["median_7"] = overall_df.apply(get_median_7, axis = 1, last_index = last_index)
        test_feats["median_30"] = overall_df.apply(get_median_30, axis = 1, last_index = last_index)
        test_feats["median_60"] = overall_df.apply(get_median_60, axis = 1, last_index = last_index)
        test_feats["mean_7"] = overall_df.apply(get_mean_7, axis = 1, last_index = last_index)
        test_feats["mean_30"] = overall_df.apply(get_mean_30, axis = 1, last_index = last_index)
        test_feats["mean_60"] = overall_df.apply(get_mean_60, axis = 1, last_index = last_index)

        test_feats["month"] = last_date.split("-")[1]
        test_feats["datenum"] = last_date.split("-")[2]
        test_feats["weekday"] = datetime.datetime.strptime(last_date, '%Y-%m-%d').date().weekday()
        
        prediction = get_preds(test_feats)

        overall_df[last_date] = prediction
        pred_df = pd.concat([pred_df, pd.concat([overall_df.Page, prediction], axis = 1)], axis = 1)
        
    return overall_df

In [41]:
# overall_df = get_overall_df()
overall_df = test_and_pred(overall_df)

2017-01-01
                                                Page  median_7  median_30  \
0            2NE1_zh.wikipedia.org_all-access_spider      20.0       19.5   
1             2PM_zh.wikipedia.org_all-access_spider      26.0       22.0   
2              3C_zh.wikipedia.org_all-access_spider       4.0        3.5   
3         4minute_zh.wikipedia.org_all-access_spider      16.0       16.5   
4  52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...      13.0        8.5   

   median_60     mean_7    mean_30    mean_60 month datenum  weekday  
0       18.0  19.857143  24.966667  25.716667    01      01        6  
1       18.5  28.142857  25.000000  31.433333    01      01        6  
2        4.0   6.428571  11.133333   7.650000    01      01        6  
3       13.0  15.857143  17.400000  15.433333    01      01        6  
4       10.0  16.142857  11.933333  15.400000    01      01        6  
2017-01-02


KeyboardInterrupt: 

In [33]:
overall_df.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2017-02-20,2017-02-21,2017-02-22,2017-02-23,2017-02-24,2017-02-25,2017-02-26,2017-02-27,2017-02-28,2017-03-01
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,0,0,0,0,0,0,0,0,0,0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,0,0,0,0,0,0,0,0,0,0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,0,0,0,0,0,0,0,0,0,0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,0,0,0,0,0,0,0,0,0,0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [127]:
train_melt = pd.read_csv("train_feats.csv")
train_melt.head()

,mean_7,mean_30,mean_60,median_7,median_30,median_60,date_num,weekday
0,NaN,NaN,NaN,NaN,NaN,NaN,2,2
1,NaN,NaN,NaN,NaN,NaN,NaN,2,2
2,NaN,NaN,NaN,NaN,NaN,NaN,2,2
3,NaN,NaN,NaN,NaN,NaN,NaN,2,2
4,NaN,NaN,NaN,NaN,NaN,NaN,2,2


In [151]:
pd.rolling_mean(np.diff(np.array([1,2,3])), window=2)

/Library/Python/2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pd.rolling_mean is deprecated for ndarrays and will be removed in a future version
  """Entry point for launching an IPython kernel.


array([ nan,   1.])

In [157]:
train.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0


In [ ]:
# train_melt = pd.read_csv("train_feats.csv")
key = pd.read_csv("key_trunc.csv")

train_melt = pd.merge(train_melt, key, left_on="Page", right_on="trunc_page", how="inner")
train_melt.drop("Page_x", inplace=True, axis = 1)
train_melt.drop("Page_y", inplace=True, axis = 1)

train_melt.to_csv("train_merged.csv", index = False)

In [93]:
train_merged_filtered = train_merged[["Id", "median_60_days"]]

In [111]:
sub = pd.merge(sub, train_merged_filtered, on = "Id")
sub.drop("Visits", inplace=True, axis = 1)
sub.columns = ["Id", "Visits"]

In [112]:
sub.to_csv("median_60_days.csv", index = False)

In [50]:
key.head()

,Id,trunc_page,date
0,bf4edcf969af,!vote_en.wikipedia.org_all-access_all-agents,2017-01-01
1,929ed2bf52b9,!vote_en.wikipedia.org_all-access_all-agents,2017-01-02
2,ff29d0f51d5c,!vote_en.wikipedia.org_all-access_all-agents,2017-01-03
3,e98873359be6,!vote_en.wikipedia.org_all-access_all-agents,2017-01-04
4,fa012434263a,!vote_en.wikipedia.org_all-access_all-agents,2017-01-05
